In [72]:
import pandas as pd
import numpy as np
import os
import requests
from tqdm import tqdm
from datetime import datetime, timedelta
import json



In [73]:
with open("api_key.txt", 'r') as file:
    API_KEY = file.read()

In [74]:

#SportsRadar API daily_summaries get request for a single date. Used in below loop 
def get_tennis_daily_summary(d, #date of summary in string format "yyyy-mm-dd"
                             write_json=True,
                             output_path = "data/json",
                             max_api_len = 200):
    
    d_num = d.replace("-","")
    
    
    
    url = f"https://api.sportradar.com/tennis/trial/v3/en/schedules/{d}/summaries.json"
    headers = {
            'accept'  : 'application/json',
            'x-api-key'  : API_KEY
    }
    response = requests.get(url,headers=headers)
        
    #parse response
    response_json = response.json() 
    
    response_json_temp = response_json
    
    #if length of response_json is 200, max limit may have been reached. need to append more possible data
    ind = max_api_len + 1
    while len(response_json_temp['summaries']) == max_api_len:
        url = f"https://api.sportradar.com/tennis/trial/v3/en/schedules/{d}/summaries.json?start={ind}"
        headers = {
                'accept'  : 'application/json',
                'x-api-key'  : API_KEY
        }
        response = requests.get(url,headers=headers)
            
        #parse response
        response_json_temp = response.json() 
        response_json['summaries'].extend(response_json_temp['summaries'])
        
        ind += max_api_len
    
    if write_json:
        file_path = os.path.join(output_path,f"{d_num}.json")
        with open(file_path, 'w') as json_file:
            json.dump(response_json, json_file, indent=4) 
    return response_json
    
    
def get_tennis_daily_summary_range(min_date,max_date):
    start = datetime.strptime(min_date, "%Y-%m-%d").date()
    end = datetime.strptime(max_date, "%Y-%m-%d").date()

    date_strings = [
        (start + timedelta(days=i)).isoformat()
        for i in range((end - start).days + 1)
    ]
    
    for dd in tqdm(range(len(date_strings))):
    
        d = date_strings[dd]
        temp = get_tennis_daily_summary(d, #date of summary in string format "yyyy-mm-dd"
                             write_json=True,
                             output_path = "data/json",
                             max_api_len = 200)
    

In [75]:
get_tennis_daily_summary_range("2026-01-05","2026-01-06")

100%|██████████| 2/2 [00:08<00:00,  4.48s/it]


# Compute Daily Scores for each player

In [76]:

min_date,max_date = "2026-01-01","2026-01-06"
#prepare final dataframe lists
date = []
comp_names = []
comp_categories = []
player_names = []
opponent_names = []
winner_flag = []
aces = []
breakpoints_won  = []
double_faults  = []
first_serve_points_won  = []
first_serve_successful  = []
games_won  = []
max_games_in_a_row  = []
max_points_in_a_row  = []
points_won_ = []
points_lost_ = []
points_won_from_last_10  = []
second_serve_points_won  = []
second_serve_successful  = []
service_games_won  = []
service_points_lost  = []
service_points_won  = []
tiebreaks_won  = []
total_breakpoints  = []
points_spread = []


start = datetime.strptime(min_date, "%Y-%m-%d").date()
end = datetime.strptime(max_date, "%Y-%m-%d").date()

date_strings = [
    (start + timedelta(days=i)).isoformat()
    for i in range((end - start).days + 1)
]


for dd in tqdm(range(len(date_strings))):
    
    d = date_strings[dd]
    
    d_num = d.replace("-","")
    
    if d_num + ".json" in os.listdir("data/json"):
        print(f'Using cache for date: {d}')
        #use cache
        with open(f"data/json/{d_num}.json", 'r') as json_file:
            response_json = json.load(json_file)
    else:
        #get and save request from API 
        print(f"API request for date: {d}")
        response_json = get_tennis_daily_summary(d)
          

    
    if "summaries" not in response_json.keys():
        print(response_json)
        continue

    for i in range(len(response_json['summaries'])):
        
        if response_json['summaries'][i]['sport_event_status']['status']!="closed":
            continue
        
        comp_name = response_json['summaries'][i]['sport_event']['sport_event_context']['competition']['name']
        comp_type = response_json['summaries'][i]['sport_event']['sport_event_context']['competition']['type']
        comp_category = response_json['summaries'][i]['sport_event']['sport_event_context']['category']['name']
        if ("ATP" in comp_category or "WTA" in comp_category or "United Cup" in comp_category) and "doubles" not in comp_name.lower():
        #if "doubles" not in comp_name.lower():
            #should return a list of two statistics dicts, one for each player
            try:
                competitors_stats = response_json['summaries'][i]['statistics']['totals']['competitors']
            except:
                #print(comp_name)
                #print(comp_category)
                #print(response_json['summaries'][i].keys())
                continue
            assert len(competitors_stats) == 2
                            
            for j in range(2):
                date.append(d)
                comp_names.append(comp_name)
                comp_categories.append(comp_category)
                player_names.append(competitors_stats[j]['name'])
                opponent_names.append(competitors_stats[(j+1)%2]['name'])
                
                #determine winner
                if response_json['summaries'][i]["sport_event_status"]["winner_id"] == response_json['summaries'][i]["sport_event"]["competitors"][j]['id']:
                    winner_flag.append(1)
                else:
                    winner_flag.append(0)
                    
                #append stats
                try:
                    aces.append(competitors_stats[j]['statistics']['aces'])
                except:
                    aces.append(0)
                try:
                    breakpoints_won.append(competitors_stats[j]['statistics']['breakpoints_won'])
                except:
                    breakpoints_won.append(0)
                try:
                    double_faults.append(competitors_stats[j]['statistics']['double_faults'])
                except:
                    double_faults.append(0)
                try:
                    first_serve_points_won.append(competitors_stats[j]['statistics']['first_serve_points_won'])
                except:
                    first_serve_points_won.append(0)
                try:
                    first_serve_successful.append(competitors_stats[j]['statistics']['first_serve_successful'])
                except:
                    first_serve_successful.append(0)
                try:
                    games_won.append(competitors_stats[j]['statistics']['games_won'])
                except:
                    games_won.append(0)
                try:
                    max_games_in_a_row.append(competitors_stats[j]['statistics']['max_games_in_a_row'])
                except:
                    max_games_in_a_row.append(0)
                try:
                    max_points_in_a_row.append(competitors_stats[j]['statistics']['max_points_in_a_row'])
                except:
                    max_points_in_a_row.append(0)
                try:
                    points_won = competitors_stats[j]['statistics']['points_won']
                    points_lost = competitors_stats[(j+1)%2]['statistics']['points_won']
                    
                    points_won_.append(points_won)
                    points_lost_.append(points_lost)
                    points_spread.append(points_won-points_lost)
                except:
                    points_won_.append(0)
                    points_lost_.append(0)
                    points_spread.append(0)
                try:
                    second_serve_points_won.append(competitors_stats[j]['statistics']['second_serve_points_won'])
                except:
                    second_serve_points_won.append(0)
                try:
                    second_serve_successful.append(competitors_stats[j]['statistics']['second_serve_successful'])
                except:
                    second_serve_successful.append(0)
                try:
                    service_games_won.append(competitors_stats[j]['statistics']['service_games_won'])
                except:
                    service_games_won.append(0)
                try:
                    service_points_lost.append(competitors_stats[j]['statistics']['service_points_lost'])
                except:
                    service_points_lost.append(0)
                try:
                    service_points_won.append(competitors_stats[j]['statistics']['service_points_won'])
                except:
                    service_points_won.append(0)
                try:
                    tiebreaks_won.append(competitors_stats[j]['statistics']['tiebreaks_won'])
                except:
                    tiebreaks_won.append(0)
                try:
                    total_breakpoints.append(competitors_stats[j]['statistics']['total_breakpoints'])
                except:
                    total_breakpoints.append(0)
                    
                

                
atp_wta_fantasy_df = pd.DataFrame({"match_date":date,
            "competition_name":comp_names,
            "competition_category":comp_categories,
            "player_name":player_names,
            "opponent_name":opponent_names,
            "winner_flag":winner_flag,
            "aces":aces,
            "breakpoints_won":breakpoints_won,
            "double_faults":double_faults,
            "first_serve_points_won":first_serve_points_won ,
            "first_serve_successful":first_serve_successful ,
            "games_won":games_won,
            "max_games_in_a_row":max_games_in_a_row ,
            "max_points_in_a_row":max_points_in_a_row ,
            "points_won":points_won_ ,
            "points_spread":points_spread,
            "second_serve_points_won":second_serve_points_won ,
            "second_serve_successful":second_serve_successful ,
            "service_games_won":service_games_won,
            "service_points_lost":service_points_lost ,
            "service_points_won":service_points_won ,
            "tiebreaks_won":tiebreaks_won ,
            "total_breakpoints":total_breakpoints
        })

atp_wta_fantasy_df["first_serve_win_perc"] = 100*atp_wta_fantasy_df["first_serve_points_won"]/atp_wta_fantasy_df["first_serve_successful"]
atp_wta_fantasy_df["second_serve_win_perc"] = 100*(atp_wta_fantasy_df["second_serve_points_won"]/(atp_wta_fantasy_df["second_serve_successful"] + atp_wta_fantasy_df["double_faults"]))
atp_wta_fantasy_df["breakpoint_conversion_perc"] = 100*(atp_wta_fantasy_df["breakpoints_won"]/(atp_wta_fantasy_df["total_breakpoints"]))

  0%|          | 0/6 [00:00<?, ?it/s]

Using cache for date: 2026-01-01
Using cache for date: 2026-01-02
Using cache for date: 2026-01-03
Using cache for date: 2026-01-04
Using cache for date: 2026-01-05


100%|██████████| 6/6 [00:00<00:00, 14.12it/s]

Using cache for date: 2026-01-06


In [77]:
atp_wta_fantasy_df.to_csv("2026 Raw Data.csv")

In [12]:
atp_summary_df = atp_wta_fantasy_df[atp_wta_fantasy_df["competition_category"]=="ATP"].\
        groupby("player_name").agg({"winner_flag":"sum",
        "aces":"sum",
        "breakpoints_won":"sum",
        "total_breakpoints":"sum",
        "double_faults":"sum",
        "first_serve_points_won":"sum",
        "first_serve_successful":"sum",
        "second_serve_points_won":"sum",
        "second_serve_successful":"sum",
        "points_spread":"sum"}).reset_index().rename({"winner_flag":"total_wins"},axis=1)

atp_summary_df["first_serve_win_perc"] = 100*atp_summary_df["first_serve_points_won"]/atp_summary_df["first_serve_successful"]
atp_summary_df["second_serve_win_perc"] = 100*(atp_summary_df["second_serve_points_won"]/(atp_summary_df["second_serve_successful"] + atp_summary_df["double_faults"]))
atp_summary_df["breakpoint_conversion_perc"] = 100*(atp_summary_df["breakpoints_won"]/(atp_summary_df["total_breakpoints"]))

In [13]:
atp_summary_df.to_csv("2026 Player Summaries.csv")

In [164]:
wta_summary_df = atp_wta_fantasy_df[atp_wta_fantasy_df["competition_category"]=="WTA"].\
        groupby("player_name").agg({"winner_flag":"sum",
        "aces":"sum",
        "breakpoints_won":"sum",
        "total_breakpoints":"sum",
        "double_faults":"sum",
        "first_serve_points_won":"sum",
        "first_serve_successful":"sum",
        "second_serve_points_won":"sum",
        "second_serve_successful":"sum",
        "points_spread":"sum"}).reset_index().rename({"winner_flag":"total_wins"},axis=1)

wta_summary_df["first_serve_win_perc"] = 100*wta_summary_df["first_serve_points_won"]/wta_summary_df["first_serve_successful"]
wta_summary_df["second_serve_win_perc"] = 100*(wta_summary_df["second_serve_points_won"]/(wta_summary_df["second_serve_successful"] + wta_summary_df["double_faults"]))
wta_summary_df["breakpoint_conversion_perc"] = 100*(wta_summary_df["breakpoints_won"]/(wta_summary_df["total_breakpoints"]))

In [165]:
wta_summary_df.to_csv("wta_summary_df.csv")

In [166]:
atp_wta_fantasy_df.to_csv("atp_wta_match_data_2025.csv")

In [113]:
atp_wta_fantasy_df[(atp_wta_fantasy_df.player_name.str.contains("Sinner")) & \
                    (atp_wta_fantasy_df.competition_name.str.contains("US Open"))
                ]

,match_date,competition_name,competition_category,player_name,opponent_name,aces,breakpoints_won,double_faults,first_serve_points_won,first_serve_successful,...,max_points_in_a_row,points_won,points_spread,second_serve_points_won,second_serve_successful,service_games_won,service_points_lost,service_points_won,tiebreaks_won,total_breakpoints
14254,2025-08-26,US Open Men Singles,ATP,"Sinner, Jannik","Kopriva, Vit",7,7,1,32,39,...,12,1,-73,16,28,11,20,48,0,10
14402,2025-08-28,US Open Men Singles,ATP,"Sinner, Jannik","Popyrin, Alexei",6,5,4,35,45,...,11,1,-73,22,33,13,24,57,0,9
14474,2025-08-30,US Open Men Singles,ATP,"Sinner, Jannik","Shapovalov, Denis",2,6,5,55,65,...,8,1,-73,22,40,17,33,77,0,13
14608,2025-09-01,US Open Men Singles,ATP,"Sinner, Jannik","Bublik, Alexander",8,8,1,25,31,...,7,1,-73,16,21,10,12,41,0,17
14720,2025-09-04,US Open Men Singles,ATP,"Sinner, Jannik","Musetti, Lorenzo",10,5,0,42,46,...,8,1,-73,15,29,13,18,57,0,6
14766,2025-09-05,US Open Men Singles,ATP,"Sinner, Jannik","Auger-Aliassime, Felix",11,4,4,44,58,...,6,1,-73,34,47,17,31,78,0,11
14870,2025-09-07,US Open Men Singles,ATP,"Sinner, Jannik","Alcaraz, Carlos",2,1,4,37,54,...,8,1,-73,28,54,12,47,65,0,1


In [78]:
def get_atp_wta_fantasy_data(min_date, max_date):


    #prepare final dataframe lists
    date_ = []
    gender = []
    comp_names = []
    comp_categories = []
    player_names = []
    opponent_names = []
    aces = []
    breakpoints_won  = []
    double_faults  = []
    first_serve_points_won  = []
    first_serve_successful  = []
    games_won  = []
    max_games_in_a_row  = []
    max_points_in_a_row  = []
    points_won  = []
    points_won_from_last_10  = []
    second_serve_points_won  = []
    second_serve_successful  = []
    service_games_won  = []
    service_points_lost  = []
    service_points_won  = []
    tiebreaks_won  = []
    total_breakpoints  = []
    
    start = datetime.strptime(min_date, "%Y-%m-%d").date()
    end = datetime.strptime(max_date, "%Y-%m-%d").date()

    date_strings = [
        (start + timedelta(days=i)).isoformat()
        for i in range((end - start).days + 1)
    ]


    for i in tqdm(range(len(date_strings))):
        
        d = date_strings[i]
        
        #get request
        
        url = f"https://api.sportradar.com/tennis/trial/v3/en/schedules/{d}/summaries.json"
        headers = {
            'accept'  : 'application/json',
            'x-api-key'  : API_KEY
        }
        response = requests.get(url,headers=headers)
        
        #parse response
        response_json = response.json()
        
        date_num = d.replace("-","")
        
        file_path = f"data/json/{date_num}.json"
        with open(file_path, 'w') as json_file:
            json.dump(response.json(), json_file, indent=4) 
        
        if "summaries" not in response_json.keys():
            print(response_json)
            continue

        for i in range(len(response_json['summaries'])):
            comp_name = response_json['summaries'][i]['sport_event']['sport_event_context']['competition']['name']
            comp_category = response_json['summaries'][i]['sport_event']['sport_event_context']['category']['name']
            if ("ATP" in comp_category or "WTA" in comp_category) and "doubles" not in comp_name.lower():
                #should return a list of two statistics dicts, one for each player
                try:
                    competitors_stats = response_json['summaries'][i]['statistics']['totals']['competitors']
                except:
                    #print(comp_name)
                    #print(comp_category)
                    #print(response_json['summaries'][i].keys())
                    continue
                assert len(competitors_stats) == 2
                
                if "/" in competitors_stats[0]['name']:
                    continue
                                
                for j in range(2):
                    date_.append(d)
                    comp_names.append(comp_name)
                    comp_categories.append(comp_category)
                    player_names.append(competitors_stats[j]['name'])
                    opponent_names.append(competitors_stats[(j+1)%2]['name'])
                    try:
                        aces.append(competitors_stats[j]['statistics']['aces'])
                    except:
                        aces.append(0)
                    try:
                        breakpoints_won.append(competitors_stats[j]['statistics']['breakpoints_won'])
                    except:
                        breakpoints_won.append(0)
                    try:
                        double_faults.append(competitors_stats[j]['statistics']['double_faults'])
                    except:
                        double_faults.append(0)
                    try:
                        first_serve_points_won.append(competitors_stats[j]['statistics']['first_serve_points_won'])
                    except:
                        first_serve_points_won.append(0)
                    try:
                        first_serve_successful.append(competitors_stats[j]['statistics']['first_serve_successful'])
                    except:
                        first_serve_successful.append(0)
                    try:
                        games_won.append(competitors_stats[j]['statistics']['games_won'])
                    except:
                        games_won.append(0)
                    try:
                        max_games_in_a_row.append(competitors_stats[j]['statistics']['max_games_in_a_row'])
                    except:
                        max_games_in_a_row.append(0)
                    try:
                        max_points_in_a_row.append(competitors_stats[j]['statistics']['max_points_in_a_row'])
                    except:
                        max_points_in_a_row.append(0)
                    try:
                        points_won.append(competitors_stats[j]['statistics']['points_won'])
                    except:
                        points_won.append(0)
                    try:
                        second_serve_points_won.append(competitors_stats[j]['statistics']['second_serve_points_won'])
                    except:
                        second_serve_points_won.append(0)
                    try:
                        second_serve_successful.append(competitors_stats[j]['statistics']['second_serve_successful'])
                    except:
                        second_serve_successful.append(0)
                    try:
                        service_games_won.append(competitors_stats[j]['statistics']['service_games_won'])
                    except:
                        service_games_won.append(0)
                    try:
                        service_points_lost.append(competitors_stats[j]['statistics']['service_points_lost'])
                    except:
                        service_points_lost.append(0)
                    try:
                        service_points_won.append(competitors_stats[j]['statistics']['service_points_won'])
                    except:
                        service_points_won.append(0)
                    try:
                        tiebreaks_won.append(competitors_stats[j]['statistics']['tiebreaks_won'])
                    except:
                        tiebreaks_won.append(0)
                    try:
                        total_breakpoints.append(competitors_stats[j]['statistics']['total_breakpoints'])
                    except:
                        total_breakpoints.append(0)
                    
    atp_wta_fantasy_df = pd.DataFrame({"match_date":date_,
                "competition_name":comp_names,
                "competition_category":comp_categories,
                "player_name":player_names,
                "opponent_name":opponent_names,
                "aces":aces,
                "breakpoints_won":breakpoints_won,
                "double_faults":double_faults,
                "first_serve_points_won":first_serve_points_won ,
                "first_serve_successful":first_serve_successful ,
                "games_won":games_won ,
                "max_games_in_a_row":max_games_in_a_row ,
                "max_points_in_a_row":max_points_in_a_row ,
                "points_won":points_won ,
                "second_serve_points_won":second_serve_points_won ,
                "second_serve_successful":second_serve_successful ,
                "service_games_won":service_games_won ,
                "service_points_lost":service_points_lost ,
                "service_points_won":service_points_won ,
                "tiebreaks_won":tiebreaks_won ,
                "total_breakpoints":total_breakpoints })
    return atp_wta_fantasy_df
    


In [55]:
atp_wta_historical_df_2025 = get_atp_wta_fantasy_data("2025-01-01","2025-12-19")

 43%|████▎     | 153/353 [10:02<10:47,  3.24s/it]

{'message': 'Limit Exceeded'}


 44%|████▎     | 154/353 [10:03<08:13,  2.48s/it]

{'message': 'Limit Exceeded'}


 44%|████▍     | 155/353 [10:04<06:26,  1.95s/it]

...

100%|█████████▉| 352/353 [12:21<00:00,  1.52it/s]

{'message': 'Limit Exceeded'}


100%|██████████| 353/353 [12:21<00:00,  2.10s/it]

{'message': 'Too Many Requests'}


ValueError: All arrays must be of the same length

In [ ]:
#TODO: VERYIFY ADELAIDE IS IN DATA (MDISON KEYS WON)

In [123]:
pd.DataFrame({"competition_name":comp_names,
              "competition_cateogory":comp_categories,
              "player":player_names,
              "aces":aces,
              "breakpoints_won":breakpoints_won})

,competition_name,competition_cateogory,player,aces,breakpoints_won
0,Next Gen ATP Finals Men Singles,ATP,"Prizmic, Dino",14,1
1,Next Gen ATP Finals Men Singles,ATP,"Basavareddy, Nishesh",2,3
2,Next Gen ATP Finals Men Singles,ATP,"Blockx, Alexander",17,3
3,Next Gen ATP Finals Men Singles,ATP,"Engel, Justin",4,0
4,Next Gen ATP Finals Men Singles,ATP,"Tien, Learner",4,8
5,Next Gen ATP Finals Men Singles,ATP,"Jodar, Rafael",3,5
6,Next Gen ATP Finals Men Singles,ATP,"Landaluce, Martin",4,2
7,Next Gen ATP Finals Men Singles,ATP,"Budkov Kjaer, Nicolai",9,5


In [13]:
df = pd.read_csv("/Users/alexbraksator/Downloads/fantasy_tennis.csv")

In [18]:
df.columns

Index(['Unnamed: 0', 'team_abbr', 'win', 'first_serve_aces',
       'second_serve_aces', 'first_serve_in', 'first_serve_attempts',
       'second_serve_in', 'second_serve_attempts', 'first_serve_points_won',
       'second_serve_points_won', 'spread_points', 'break_point_opportunities',
       'break_point_saved', 'break_point_faced', 'total_points',
       'first_serve_percent', 'first_serve_won_percent',
       'second_serve_won_percent', 'break_point_saved_percent',
       'total_matches'],
      dtype='object')

In [22]:
rank_direction = {
    "win": True,
    "first_serve_aces": True,
    'second_serve_aces':True, 
    'first_serve_in':True, 
    'first_serve_attempts':True,
       'second_serve_in':True, 
       'second_serve_attempts':True, 
       'first_serve_points_won':True,
       'second_serve_points_won':True, 
       'spread_points':False, 
       'break_point_opportunities':True,
       'break_point_saved':True, 
       'break_point_faced':True, 
       'total_points':True,
       'first_serve_percent':True, 
       'first_serve_won_percent':True,
       'second_serve_won_percent':True, 
       'break_point_saved_percent':True,
       'total_matches':True
}

In [23]:
ranked_df = df.copy()

for col, high_is_good in rank_direction.items():
    ranked_df[col + "_Rank"] = df[col].rank(
        ascending=not high_is_good,  # ascending=False if higher is better
        method="dense"               # optional: 'min', 'dense', 'average', etc.
    )

In [45]:
ranked_df["composite"] = ranked_df[["win_Rank",
           "first_serve_aces_Rank",
           "second_serve_aces_Rank",
           "first_serve_percent_Rank",
           "first_serve_won_percent_Rank",
           "second_serve_won_percent_Rank",
           "spread_points_Rank",
           "break_point_saved_percent_Rank",
           "break_point_opportunities_Rank"
           ]].sum(axis=1)

In [52]:
ranked_df[["team_abbr", "win_Rank",
           "first_serve_aces_Rank",
           "second_serve_aces_Rank",
           "first_serve_percent_Rank",
           "first_serve_won_percent_Rank",
           "second_serve_won_percent_Rank",
           "spread_points_Rank",
           "break_point_saved_percent_Rank",
           "break_point_opportunities_Rank",
           "composite"
           ]].sort_values("composite",ascending=True).head(20)

,team_abbr,win_Rank,first_serve_aces_Rank,second_serve_aces_Rank,first_serve_percent_Rank,first_serve_won_percent_Rank,second_serve_won_percent_Rank,spread_points_Rank,break_point_saved_percent_Rank,break_point_opportunities_Rank,composite
73,Aryna Sabalenka,1.0,5.0,9.0,253.0,46.0,39.0,2.0,18.0,2.0,375.0
505,Taylor Townsend,22.0,30.0,17.0,123.0,108.0,50.0,53.0,16.0,75.0,494.0
22,Amanda Anisimova,5.0,17.0,3.0,243.0,95.0,61.0,8.0,65.0,8.0,505.0
162,Elena Rybakina,3.0,1.0,2.0,427.0,1.0,43.0,3.0,23.0,6.0,509.0
252,Jessica Pegula,3.0,19.0,20.0,295.0,68.0,41.0,4.0,75.0,1.0,526.0
337,Madison Keys,8.0,12.0,18.0,141.0,66.0,154.0,12.0,99.0,16.0,526.0
153,Ekaterina Alexandrova,4.0,4.0,12.0,249.0,36.0,170.0,5.0,46.0,3.0,529.0
227,Iga Swiatek,2.0,10.0,15.0,302.0,29.0,54.0,1.0,112.0,4.0,529.0
84,Belinda Bencic,8.0,59.0,20.0,161.0,92.0,105.0,13.0,64.0,22.0,544.0
399,Mirra Andreeva,7.0,26.0,14.0,337.0,49.0,98.0,6.0,41.0,21.0,599.0


In [55]:
ranked_df.to_csv("/Users/alexbraksator/Downloads/WTA_draft.csv")